# Arrays and Images
- toc: true 
- badges: true
- comments: true
- categories: [jupyter]
- title: Arrays and Images

## PNG to Gif

In [2]:
import javax.imageio.ImageIO;
import java.io.File;
import java.io.IOException;
import java.awt.image.BufferedImage;

import java.util.Scanner;  // Import the Scanner class

public class ImageIOTest {
    // making the user input
    private static String inputName() {
        Scanner imageName = new Scanner(System.in);  // Create a Scanner object
        System.out.print("Enter an option number (1-3): ");
        int option = imageName.nextInt(); // storing the option as the variable
        System.out.println(option);
        String name = ""; //Creating a string variable for the name of the image
        if(option == 1){
            name = "MonaLisa";
        }
        else if(option == 2){
            name = "AmongUs";
        }
        else if(option == 3){
            name = "Spiderman";
        }
        else{
            System.out.print("Unexpected choice, try again.");
        }
        System.out.println("You chose the option: " + name);
        return name;
    }

    public static void main( String[] args ){
       BufferedImage img = null;  // buffer type       

        try {
            
            // Name of file and directories
            String name = inputName();
            String in = "images/";
            String out = "images/tmp/";

            // Either use URL or File for reading image using ImageIO
            File imageFile = new File(in + name + ".png");
            img = ImageIO.read(imageFile);  // set buffer of image data

            // ImageIO Image write to gif in Java
            // Documentation https://docs.oracle.com/javase/tutorial/2d/images/index.html
            ImageIO.write(img, "gif", new File(out + name + ".gif") );  // write buffer to gif

        } catch (IOException e) {
              e.printStackTrace();
        }
        System.out.println("The gif has been created successfully");
    }
}
ImageIOTest.main(null);

Enter an option number (1-3): 1
You chose the option: MonaLisa
The gif has been created successfully


## Scales and Ascii

In [23]:
import java.awt.Color;
import java.awt.image.BufferedImage;
import java.awt.Image;
import java.awt.Graphics2D;

import java.io.File;
import java.io.FileWriter;
import java.io.IOException;
import java.io.PrintWriter;
import java.nio.file.Files;
import java.nio.file.Path;
import java.nio.file.Paths;

import javax.imageio.stream.ImageOutputStream;
import javax.imageio.stream.ImageInputStream;
import javax.imageio.metadata.IIOMetadata;
import javax.imageio.IIOImage;
import javax.imageio.ImageIO;
import javax.imageio.ImageWriteParam;
import javax.imageio.ImageWriter;
import javax.imageio.ImageReader;
import javax.imageio.ImageTypeSpecifier;

public class Pics {
    private final String inDir = "images/"; // location of images
    private final String outDir = "images/tmp/";  // location of created files
    private final String grayDir = "images/tmp/grey/";
    private final String greenDir = "images/tmp/green/";
    private final String blueDir = "images/tmp/blue/";
    private final String redDir =  "images/tmp/red";
    private String inFile;
    private String resizedFile;
    private String blueFile;
    private String greenFile;
    private String redFile;
    private String greyFile;
    private String asciiFile;
    private String ext;   // extension of file
    private long bytes;
    private int width;
    private int height;

    // Constructor obtains attributes of picture
    public Pics(String name, String ext) {
        this.ext = ext;
        this.inFile = this.inDir + name + "." + ext;
        this.resizedFile = this.outDir + name + "." + ext;
        this.asciiFile = this.outDir + name + ".txt";
        this.setStats();
        this.blueFile = this.outDir + name + "BlueScale." + ext;
        this.greenFile = this.outDir + name + "GreenScale." + ext;
        this.redFile = this.outDir + name + "RedScale." + ext;
        this.greyFile = this.outDir + name + "GreyScale." + ext;
    }

    

    
    // An image contains metadata, namely size, width, and height
    public void setStats() {
        BufferedImage img;
        try {
            Path path = Paths.get(this.inFile);
            this.bytes = Files.size(path);
            img = ImageIO.read(new File(this.inFile));
            this.width = img.getWidth();
            this.height = img.getHeight();
        } catch (IOException e) {
        }
    }

    // Console print of data
    public void printStats(String msg) {
        System.out.println(msg + ": " + this.bytes + " " + this.width + "x" + this.height + "  " + this.inFile);
    }

    // Convert scaled image into buffered image
    public static BufferedImage convertToBufferedImage(Image img) {

        // Create a buffered image with transparency
        BufferedImage bi = new BufferedImage(
                img.getWidth(null), img.getHeight(null),
                BufferedImage.TYPE_INT_ARGB);

        // magic?
        Graphics2D graphics2D = bi.createGraphics();
        graphics2D.drawImage(img, 0, 0, null);
        graphics2D.dispose();

        return bi;
    }
    
    // Scale or reduce to "scale" percentage provided
    public void resize(int scale) {
        BufferedImage img = null;
        Image resizedImg = null;  

        int width = (int) (this.width * (scale/100.0) + 0.5);
        int height = (int) (this.height * (scale/100.0) + 0.5);

        try {
            // read an image to BufferedImage for processing
            img = ImageIO.read(new File(this.inFile));  // set buffer of image data
            // create a new BufferedImage for drawing
            resizedImg = img.getScaledInstance(width, height, Image.SCALE_SMOOTH);
        } catch (IOException e) {
            return;
        }

        try {
            ImageIO.write(convertToBufferedImage(resizedImg), this.ext, new File(resizedFile));
        } catch (IOException e) {
            return;
        }
        
        this.inFile = this.resizedFile;  // use scaled file vs original file in Class
        this.setStats();
    }
    
    // convert every pixel to an ascii character (ratio does not seem correct)
    public void convertToAscii() {
        BufferedImage img = null;
        PrintWriter asciiPrt = null;
        FileWriter asciiWrt = null;

        try {
            File file = new File(this.asciiFile);
            Files.deleteIfExists(file.toPath()); 
        } catch (IOException e) {
            System.out.println("Delete File error: " + e);
        }

        try {
            asciiPrt = new PrintWriter(asciiWrt = new FileWriter(this.asciiFile, true));
        } catch (IOException e) {
            System.out.println("ASCII out file create error: " + e);
        }

        try {
            img = ImageIO.read(new File(this.inFile));
        } catch (IOException e) {
        }

        for (int i = 0; i < img.getHeight(); i++) {
            for (int j = 0; j < img.getWidth(); j++) {
                Color col = new Color(img.getRGB(j, i));
                double pixVal = (0.01+(0.3*(col.getRed()/255.0)))+(0.01+(0.59*(col.getGreen()/255.0))) + (0.01+(0.11*(col.getBlue()/255.0)));
                try {
                    asciiPrt.print(asciiChar(pixVal));
                    asciiPrt.flush();
                    asciiWrt.flush();
                } catch (Exception ex) {
                }
            }
            try {
                asciiPrt.println("");
                asciiPrt.flush();
                asciiWrt.flush();
            } catch (Exception ex) {
            }
        }
    }

    public void greyscale() {
        BufferedImage img = null;
        BufferedImage greyImg = null;  

        try {
            // read an image to BufferedImage for processing
            img = ImageIO.read(new File(this.inFile));  // set buffer of image data
            greyImg = img;
            // create a new BufferedImage for drawing
            for (int i = 0; i < img.getHeight(); i++) {
                for (int j = 0; j < img.getWidth(); j++) {
                    Color co = new Color(img.getRGB(j, i));
                    int red = (int) (co.getRed() * 0.299);
                    int green = (int) (co.getGreen() * 0.587);
                    int blue = (int) (co.getBlue() * 0.114);
                    int total = red + green + blue;
                    Color newColor = new Color(total, total, total);
                    greyImg.setRGB(j, i, newColor.getRGB());
                }
            }

        } catch (IOException e) {
            return;
        }

        try {
            ImageIO.write(convertToBufferedImage(greyImg), this.ext, new File(greyFile));
        } catch (IOException e) {
            return;
        }
        
        this.inFile = this.greyFile;  // use scaled file vs original file in Class
        this.setStats();
    }

    public void redscale() {
        BufferedImage img = null;
        BufferedImage redImg = null;  

        try {
            // read an image to BufferedImage for processing
            img = ImageIO.read(new File(this.inFile));  // set buffer of image data
            redImg = img;
            // create a new BufferedImage for drawing
            for (int i = 0; i < img.getHeight(); i++) {
                for (int j = 0; j < img.getWidth(); j++) {
                    Color co = new Color(img.getRGB(j, i));
                    int red = (int) co.getRed();
                    int green = 0;
                    int blue = 0;
                    Color newColor = new Color(red, green, blue);
                    redImg.setRGB(j, i, newColor.getRGB());
                }
            }

        } catch (IOException e) {
            return;
        }

        try {
            ImageIO.write(convertToBufferedImage(redImg), this.ext, new File(redFile));
        } catch (IOException e) {
            return;
        }
        
        this.inFile = this.redFile;  // use scaled file vs original file in Class
        this.setStats();
    }

    public void greenscale() {
        BufferedImage img = null;
        BufferedImage greenImg = null;  

        try {
            // read an image to BufferedImage for processing
            img = ImageIO.read(new File(this.inFile));  // set buffer of image data
            greenImg = img;
            // create a new BufferedImage for drawing
            for (int i = 0; i < img.getHeight(); i++) {
                for (int j = 0; j < img.getWidth(); j++) {
                    Color co = new Color(img.getRGB(j, i));
                    int red = 0;
                    int green = (int) co.getGreen();
                    int blue = 0;
                    Color newColor = new Color(red, green, blue);
                    greenImg.setRGB(j, i, newColor.getRGB());
                }
            }

        } catch (IOException e) {
            return;
        }

        try {
            ImageIO.write(convertToBufferedImage(greenImg), this.ext, new File(greenFile));
        } catch (IOException e) {
            return;
        }
        
        this.inFile = this.greenFile;  // use scaled file vs original file in Class
        this.setStats();
    }

    public void bluescale() {
        BufferedImage img = null;
        BufferedImage blueImg = null;  

        try {
            // read an image to BufferedImage for processing
            img = ImageIO.read(new File(this.inFile));  // set buffer of image data
            blueImg = img;
            // create a new BufferedImage for drawing
            for (int i = 0; i < img.getHeight(); i++) {
                for (int j = 0; j < img.getWidth(); j++) {
                    Color co = new Color(img.getRGB(j, i));
                    int red = (int) (co.getRed() * 0.299);
                    int green = (int) (co.getGreen() * 0.587);
                    int blue = (int) (co.getBlue() * 0.114);
                    int total = red + green + blue;
                    Color newColor = new Color(0, 0, total);
                    blueImg.setRGB(j, i, newColor.getRGB());
                }
            }

        } catch (IOException e) {
            return;
        }

        try {
            ImageIO.write(convertToBufferedImage(blueImg), this.ext, new File(blueFile));
        } catch (IOException e) {
            return;
        }
        
        this.inFile = this.blueFile;  // use scaled file vs original file in Class
        this.setStats();
    }
 
	private static int getClosetNumberOfTarget(double[] arrayList, double value) {
 
        double current = Double.MAX_VALUE;
        int index = 0;
        for (int i = 0; i < arrayList.length; i++) {
            if (Math.abs(value - arrayList[i]) < current) {
                current = Math.abs(value - arrayList[i]);
                index = i;
            }

        }
    return arrayList.length - index;
}

    // conversion table, there may be better out there ie https://www.billmongan.com/Ursinus-CS173-Fall2020/Labs/ASCIIArt
    public String asciiChar(double g) {
        // The lookup table for all of the chars that can be used
        String GRAYSCALE_CHARS  = "@#BW8gMN6&9RQqE0d$DbpmOa5AH2GZeKPS3X4U%kwohFVyzIsCu{xfn}tJTi17c=jYL?l[]v<>+|)r(/\\*_!;^~\":,-'.`";
    // The lookup table of the brightness of each char, from 0 to 1
        double[] GRAYSCALE_VALUES = {0, 0.07863, 0.08205, 0.1316, 0.1453, 0.147, 0.1504, 0.1641, 0.1761, 0.1761, 0.1778, 0.2017, 0.2154, 0.2274, 0.2274, 0.2291, 0.2427, 0.2598, 0.2615, 0.2684, 0.2752, 0.2769, 0.2889, 0.2923, 0.294, 0.2974, 0.3026, 0.3077, 0.3077, 0.3094, 0.3162, 0.3197, 0.3333, 0.3333, 0.3385, 0.3538, 0.3607, 0.3795, 0.3846, 0.4017, 0.4034, 0.4068, 0.4291, 0.4325, 0.4444, 0.4598, 0.4632, 0.4667, 0.4752, 0.4855, 0.5077, 0.5128, 0.5128, 0.5128, 0.5179, 0.5214, 0.5231, 0.535, 0.535, 0.5368, 0.5385, 0.5436, 0.5624, 0.5624, 0.5624, 0.5641, 0.5778, 0.5829, 0.5846, 0.5863, 0.5863, 0.5915, 0.6017, 0.6068, 0.641, 0.6581, 0.6889, 0.694, 0.6974, 0.7333, 0.7333, 0.7402, 0.7675, 0.7744, 0.7863, 0.8068, 0.8342, 0.8427, 0.8598, 0.894, 0.947, 0.959, 0.9675, 1};
    
        //find the closest value in the grayscale_value find its length in the list and find the compared length to the chars
        String str = " ";
        if(getClosetNumberOfTarget(GRAYSCALE_VALUES, g) == 94){
            str = " ";
        }
        else{
            str = "" + GRAYSCALE_CHARS.charAt(getClosetNumberOfTarget(GRAYSCALE_VALUES, g)) + GRAYSCALE_CHARS.charAt(getClosetNumberOfTarget(GRAYSCALE_VALUES, g))+ GRAYSCALE_CHARS.charAt(getClosetNumberOfTarget(GRAYSCALE_VALUES, g));
        }
        
        // String str = " ";
        // if (g >= 240) {
        //     str = " ";
        // } else if (g >= 210) {
        //     str = ".";
        // } else if (g >= 190) {
        //     str = "*";
        // } else if (g >= 170) {
        //     str = "+";
        // } else if (g >= 120) {
        //     str = "^";
        // } else if (g >= 110) {
        //     str = "&";
        // } else if (g >= 80) {
        //     str = "8";
        // } else if (g >= 60) {
        //     str = "#";
        // } else {
        //     str = "@";
        // }
        return str;
    }

    // tester/driver
    public static void main(String[] args) throws IOException {
        Pics monaLisa = new Pics("MonaLisa", "png");
        monaLisa.printStats("Original");
        monaLisa.resize(33);
        monaLisa.printStats("Scaled");
        monaLisa.convertToAscii();
        //monaLisa.bluescale();
        //monaLisa.redscale();
        //monaLisa.greenscale();
        //monaLisa.greyscale();

        Pics spiderman = new Pics("Spiderman", "png");
        spiderman.printStats("Original");
        spiderman.resize(33);
        spiderman.printStats("Scaled");
        spiderman.convertToAscii();
        //spiderman.bluescale();
        //spiderman.redscale();
        //spiderman.greenscale();
        //spiderman.greyscale();

        Pics amongUs = new Pics("AmongUs", "png");
        amongUs.printStats("Original");
        amongUs.resize(33);
        amongUs.printStats("Scaled");
        amongUs.convertToAscii();
        //amongUs.bluescale();
        //amongUs.redscale();
        //amongUs.greenscale();
        //amongUs.greyscale();
    }
}
Pics.main(null);

Original: 711262 512x765  images/MonaLisa.png
Scaled: 106762 169x252  images/tmp/MonaLisa.png
Original: 207325 820x726  images/Spiderman.png
Scaled: 65495 271x240  images/tmp/Spiderman.png
Original: 453797 900x506  images/AmongUs.png
Scaled: 93374 297x167  images/tmp/AmongUs.png
Original: 12833 480x279  images/Batman.png
Scaled: 6503 158x92  images/tmp/Batman.png
